# Naming generated clusters using gemini

In [25]:
import pandas as pd
import google.genai as genai
import time
import os
import json
api_key = os.getenv("GEMINI_API_KEY")


In [26]:
num_clusters = 120
reduction_method = "tsne"
df = pd.read_csv(f'data/processed/plotly/starmap_data_{reduction_method}_{num_clusters}.csv')
df.head()

,id,title,description,thumbnail,youtube_url,cluster_id,x,y,z
0,fandom_104508,Akidearest,"Agnes Yulo Diego (born: February 16, 1993 (199...",https://static.wikia.nocookie.net/youtube/imag...,https://www.youtube.com/@akidearest,0,-12.302430,-26.505335,-18.164293
1,fandom_435287,Kitakara Tome,"Akari Gin, better known as Kitakara Tome, prev...",https://static.wikia.nocookie.net/youtube/imag...,https://www.youtube.com/@KitakaraTome,0,-5.289271,-19.342413,-11.290625
2,fandom_542475,Brazilian Girl,Brazilian Girl [Age 13] is a YouTuber in the G...,https://static.wikia.nocookie.net/youtube/imag...,https://www.youtube.com/@Brazilian_Girll,0,2.113042,-17.812296,26.449568
3,fandom_451905,Leni Robredo,"Maria Leonor ""Leni"" Robredo (née Gerona; born:...",https://static.wikia.nocookie.net/youtube/imag...,https://www.youtube.com/channel/UCvlZWzCZfRb1P...,0,-0.553885,-32.333420,26.564972
4,fandom_436816,Dr. Dana Figura,"Dr. Dana Figura, also known as FootDocDana, is...",https://static.wikia.nocookie.net/youtube/imag...,https://www.youtube.com/channel/UCPc9yOXlAFXcL...,0,-20.156288,-35.300457,34.035706


## Get new df, with descriptions cut to 3000 characters and cluster groups only

In [27]:
df_gemini = df[["title", "description", "cluster_id"]]
df_gemini.head()

,title,description,cluster_id
0,Akidearest,"Agnes Yulo Diego (born: February 16, 1993 (199...",0
1,Kitakara Tome,"Akari Gin, better known as Kitakara Tome, prev...",0
2,Brazilian Girl,Brazilian Girl [Age 13] is a YouTuber in the G...,0
3,Leni Robredo,"Maria Leonor ""Leni"" Robredo (née Gerona; born:...",0
4,Dr. Dana Figura,"Dr. Dana Figura, also known as FootDocDana, is...",0


## Reduce description down to 3000

Ran out of quota. Doing a sloppy way instead

In [28]:
df_gemini['description'] = df_gemini['description'].str[:3000]

/var/folders/w7/kp2_hvs50r57qw4_9vn5k0tr0000gn/T/ipykernel_64478/214877023.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gemini['description'] = df_gemini['description'].str[:3000]


Save to a file, then give it to gemini pro

In [29]:
df_gemini.to_csv(f'data/processed/plotly/starmap_data_{reduction_method}_trimmed_{num_clusters}.csv', index=False)

In [30]:
labels = {
    0: "Lifestyle & Personal Vlogs",
    1: "Compilations, Memes & Shorts",
    2: "Music Remixes, Covers & Mashups",
    3: "History, Geography & Educational",
    4: "Family Vlogs & Personal Stories",
    5: "Anime Reviews & Podcasts",
    6: "Roblox Gaming & Gameplay",
    7: "Mario, Nintendo & Speedrunning",
    8: "Commentary & Vlogs",
    9: "Russian Language Vlogs & Music",
    10: "Pop Singers & Songwriters",
    11: "Gacha Life & Animation",
    12: "Korean & Chinese Culture/Vlogs",
    13: "Cooking, Food & Recipes",
    14: "Aviation & Airlines",
    15: "Tech Reviews & Keyboards",
    16: "Spongebob & Cartoon Parodies",
    17: "Minecraft Gameplay & Tutorials",
    18: "Variety Gaming & Stream Stats",
    19: "Minecraft SMPs & Roleplay",
    20: "Metal, Metalcore & Alternative Music",
    21: "VTubers (Nijisanji & Indie)",
    22: "Comedy Skits & Podcasts",
    23: "Hololive & Japanese VTubers",
    24: "Cats, Dogs & Pet Vlogs",
    25: "Political Commentary & Video Essays",
    26: "Pop Music & Chart Toppers",
    27: "Police, Law Enforcement & Real Life",
    28: "Retail Brands & Product Commercials",
    29: "Religion, Theology & Skepticism",
    30: "Female Pop Artists & Divas",
    31: "Instrumental Covers & Guitar",
    32: "2D Animation & Art Community",
    33: "TikTok Trends & Influencers",
    34: "Tech Companies & Digital Services",
    35: "Hip-Hop History & Celebrity News",
    36: "Fast Food Chains & Restaurants",
    37: "Twitch Streamers & VTubers",
    38: "Horror Stories & Creepypasta",
    39: "Teen Vlogs & Commentary",
    40: "Brazilian & Portuguese Content",
    41: "Sonic the Hedgehog Fan Content",
    42: "Ice Hockey (NHL)",
    43: "Variety Gaming & Let's Plays",
    44: "J-Pop, Anime Music & Vocaloid",
    45: "Top Lists, Facts & Trivia",
    46: "Trains, Model Railroads & Cars",
    47: "Classic Rock & Hard Rock Bands",
    48: "VTuber Music & Covers",
    49: "80s/90s Pop & Rock Bands",
    50: "TikTok Compilations & Shorts",
    51: "Minecraft Competitions (MCC)",
    52: "Grand Theft Auto (GTA) Gaming",
    53: "DIY, Makers & Experiments",
    54: "US Sports (MLB, NFL)",
    55: "Retro Gaming & Tech History",
    56: "3D Pen & Creative Art",
    57: "Animated Stories & Family Content",
    58: "Science & Educational Explainers",
    59: "Five Nights at Freddy's (FNAF)",
    60: "Corporate Business & Gyms",
    61: "European Gaming & Minecraft",
    62: "Object Shows (BFDI)",
    63: "Software & Tech Archives",
    64: "Beauty, Makeup & Nail Art",
    65: "Art, Drawing & VTuber Design",
    66: "Memes & Shitposting",
    67: "Global Football (Soccer) & Rugby",
    68: "Gacha Life Stories & Drama",
    69: "Travel Vlogs & Parkour",
    70: "Electronic & Dance Music (EDM)",
    71: "Nijisanji & VTuber Groups",
    72: "Kids Toys & Nursery Rhymes",
    73: "Educational Tutorials & Math",
    74: "LEGO Stop Motion & Brickfilms",
    75: "Hip-Hop, Rap & Trap",
    76: "Roblox Developers & Skits",
    77: "UK Rap, Grime & Culture",
    78: "FIFA & UK Football Culture",
    79: "Call of Duty & FPS Gaming",
    80: "Animation Glitches & Commentary",
    81: "Independent VTubers",
    82: "Voice Acting, Dubs & Parodies",
    83: "Pokemon & Nintendo Gaming",
    84: "Government & International Orgs",
    85: "TV Networks & Broadcasting",
    86: "NCS & Trap Music",
    87: "BookTube & Theater",
    88: "Team Fortress 2 (TF2)",
    89: "Indian Vlogs & Desi Content",
    90: "Movie Reviews, Comics & Star Wars",
    91: "Rants, Commentary & Drama",
    92: "Sketch Comedy & Smosh",
    93: "Neurodiversity & Self-Improvement",
    94: "Internet Personalities & Vloggers",
    95: "Tech, VR & Coding",
    96: "Music Uploads & EDM",
    97: "Minecraft PvP & Hypixel",
    98: "News Agencies & Journalism",
    99: "Social Commentary & Video Essays",
    100: "Thomas the Tank Engine Fans",
    101: "Government PSAs & Non-Profits",
    102: "Mapping & Geopolitics",
    103: "Comedy, Actors & Talk Shows",
    104: "Geometry Dash Levels",
    105: "Pokemon, Minecraft & Variety",
    106: "YouTuber News & Drama",
    107: "Local TV News & Stations",
    108: "YouTube Shorts Creators",
    109: "Middle Eastern & Arabic Vlogs",
    110: "Countryballs & Mapping",
    111: "US Politics & Politicians",
    112: "Plants vs. Zombies & Gaming",
    113: "VTubers (Anime & Gaming)",
    114: "Fortnite Competitive & FaZe",
    115: "Family Channels & Sibling Vlogs",
    116: "Animation Memes & Art Vlogs",
    117: "Female Gamers & Streamers",
    118: "Hip-Hop Mixtapes & Rappers",
    119: "Friday Night Funkin' (FNF) Mods"
}

Open the csv file, replace the cluster ID with a string name, then save as a new file

In [31]:
df_combined = pd.read_csv(f'data/processed/plotly/starmap_data_{reduction_method}_{num_clusters}.csv')

In [32]:
df_combined['cluster_name'] = df_combined['cluster_id'].map(labels)
df_combined['cluster_name'].unique()

array(['Lifestyle & Personal Vlogs', 'Compilations, Memes & Shorts',
       'Music Remixes, Covers & Mashups',
       'History, Geography & Educational',
       'Family Vlogs & Personal Stories', 'Anime Reviews & Podcasts',
       'Roblox Gaming & Gameplay', 'Mario, Nintendo & Speedrunning',
       'Commentary & Vlogs', 'Russian Language Vlogs & Music',
       'Pop Singers & Songwriters', 'Gacha Life & Animation',
       'Korean & Chinese Culture/Vlogs', 'Cooking, Food & Recipes',
       'Aviation & Airlines', 'Tech Reviews & Keyboards',
       'Spongebob & Cartoon Parodies', 'Minecraft Gameplay & Tutorials',
       'Variety Gaming & Stream Stats', 'Minecraft SMPs & Roleplay',
       'Metal, Metalcore & Alternative Music',
       'VTubers (Nijisanji & Indie)', 'Comedy Skits & Podcasts',
       'Hololive & Japanese VTubers', 'Cats, Dogs & Pet Vlogs',
       'Political Commentary & Video Essays', 'Pop Music & Chart Toppers',
       'Police, Law Enforcement & Real Life',
       'Retail Br

In [33]:
df_combined['cluster_name'] = df_combined['cluster_id'].map(labels)
df_combined.to_csv(f'data/processed/plotly/starmap_data_{reduction_method}_trimmed_{num_clusters}_labeled.csv', index=False)

Save to a new csv

In [34]:
len(df_combined['cluster_name'].unique())

120